# Performance algorithme PyCrypto 

Dans cette partie nous allons implémenter notre algorithme PyCrypto en se basant sur la définition du BestMarkowitz.
Ensuite nous allons le comparer avec les autres algorithmes.

In [33]:
pd.set_option('display.max_rows', 500)

In [1]:
import numpy as np
import pandas as pd

from universal import tools
from universal.algos import CRP


class Pycrypto(CRP):
    """
    Reference:
        https://en.wikipedia.org/wiki/Modern_portfolio_theory
    """

    PRICE_TYPE = "ratio"
    REPLACE_MISSING = False
    

    
    def __init__(self, global_sharpe=None, sharpe=None, **kwargs):
        self.global_sharpe = global_sharpe
        self.sharpe = sharpe
        self.opt_markowitz_kwargs = kwargs

    def weights(self, X, y):
        y = y.to_numpy()
        y = y.reshape(y.shape[0], 1)
        # update frequency
        freq = tools.freq(X.index)

        R = X - 1

        # calculate mean and covariance matrix and annualize them
        sigma = R.cov() * freq

        if self.sharpe:
            mu = pd.Series(np.sqrt(np.diag(sigma)), X.columns) * pd.Series(
                self.sharpe
            ).reindex(X.columns)
        elif self.global_sharpe:
            mu = pd.Series(np.sqrt(np.diag(sigma)) * self.global_sharpe, X.columns)
        else:
            mu = R.mean() * freq

        self.b = tools.opt_markowitz(mu, sigma, **self.opt_markowitz_kwargs)

        return super().weights(R)*y



In [2]:
df = pd.read_csv('année_2022_full_DF.csv', parse_dates = [0], index_col = 0)
clo = []
for e in df.columns :
    if 'Close' in e :
        clo.append(e)
df = df[clo]
df = df.dropna(axis = 1)
df = df['2022-01-31':'2022-08-07']

In [3]:
pred = pd.read_csv('X_val_pred.csv', parse_dates = [0], index_col = 0)
pred = pred.rename(columns= {'0' : 'prediction'})
T = df.copy()[[]]
T['prediction'] = pred
T = T.fillna(method = 'ffill')
T = T['2022-01-30':'2022-08-07']
T = T.replace([2,1], [1,1])

In [9]:
pred = pd.read_csv('X_val_crypto_pred_Thresholded2.csv', parse_dates = True, index_col = -1)
pred = pred.rename(columns= {'Prediction' : 'prediction'})
pred = pred.drop('Unnamed: 0', axis = 1)
pred = pred.replace([True, False], [1,1e-9]) # on met une valeur légerement diffférente de 0 pour le calcul du Sharpe
T = df.copy()[[]]
T['prediction'] = pred
T = T.fillna(method = 'ffill')
T = T['2022-01-30':'2022-08-07']
T

,prediction
Date,
2022-01-31,1.0
2022-02-01,1.0
2022-02-02,1.0
2022-02-03,1.0
2022-02-04,1.0
...,...
2022-08-03,1.0
2022-08-04,1.0
2022-08-05,1.0


In [10]:
r = Pycrypto()
res = r.weights(df,T['prediction'])

In [11]:
F = pd.DataFrame(data= r.weights(df,T['prediction']), columns = clo, index = T.index)
#F.to_csv('poids.csv')
F

,BTC-USD_Close,ETH-USD_Close,TRX-USD_Close,BNB-USD_Close,XRP-USD_Close,DOGE-USD_Close,ADA-USD_Close,SOL-USD_Close,SHIB-USD_Close,LTC-USD_Close
Date,,,,,,,,,,
2022-01-31,-6.156179e-11,4.816117e-10,0.968509,0.031486,5.845436e-07,0.000003,5.332552e-07,2.743433e-09,7.210566e-07,4.677197e-09
2022-02-01,-6.156179e-11,4.816117e-10,0.968509,0.031486,5.845436e-07,0.000003,5.332552e-07,2.743433e-09,7.210566e-07,4.677197e-09
2022-02-02,-6.156179e-11,4.816117e-10,0.968509,0.031486,5.845436e-07,0.000003,5.332552e-07,2.743433e-09,7.210566e-07,4.677197e-09
2022-02-03,-6.156179e-11,4.816117e-10,0.968509,0.031486,5.845436e-07,0.000003,5.332552e-07,2.743433e-09,7.210566e-07,4.677197e-09
2022-02-04,-6.156179e-11,4.816117e-10,0.968509,0.031486,5.845436e-07,0.000003,5.332552e-07,2.743433e-09,7.210566e-07,4.677197e-09
...,...,...,...,...,...,...,...,...,...,...
2022-08-03,-6.156179e-11,4.816117e-10,0.968509,0.031486,5.845436e-07,0.000003,5.332552e-07,2.743433e-09,7.210566e-07,4.677197e-09
2022-08-04,-6.156179e-11,4.816117e-10,0.968509,0.031486,5.845436e-07,0.000003,5.332552e-07,2.743433e-09,7.210566e-07,4.677197e-09
2022-08-05,-6.156179e-11,4.816117e-10,0.968509,0.031486,5.845436e-07,0.000003,5.332552e-07,2.743433e-09,7.210566e-07,4.677197e-09


In [12]:
df_r = df.copy()[[]]
for e in df.columns : 
    df_r['rend_cum_' + e[:3]] = df[e]/df.iloc[0][e]
# Allocation
allo = df.copy()[[]]
for e in df :
    allo['allo_' + e[:3]] = df_r['rend_cum_' + e[:3]].values*F[e].values
pos = df.copy()[[]]
for e in df.columns : 
    pos['position_' + e[:3]] = allo['allo_' + e[:3]]*10000 #montant
pos['Total'] = pos.sum(axis = 1)
pos['rendement_quotidien'] = pos['Total'].pct_change(1)
rend_cumulatif = 100*(pos['Total'][-1]/pos['Total'][0] - 1)
sharp = pos['rendement_quotidien'].mean()/pos['rendement_quotidien'].std()
sharp_anu = sharp*(365**(1/2))

In [13]:
sharp_anu

2.8022679859739505

## Sharpe pour tous les algorithmes

In [14]:
from universal.algos import *
import warnings
warnings.filterwarnings('ignore')

In [75]:
def sharp(m = 10000, k = CWMR()) : 
    '''
    Cette fonction calcule le ratio de sharpe annuel et le rendement cummulatif
    m :  montant qu'on veut investir
    k = algorithme choisi parmi :
    [ Anticor(), BAH(), BCRP(), BestMarkowitz(), BestSoFar(), BNN(),CORN(), CRP(), CWMR(), DynamicCRP(), 
    EG(), OLMAR(), ONS(),PAMR(),RMR()]
    path : chemin du dataframe des closes
    Exemple : sharp(m = 10000, k = CWMR())[0]
    >> 2.31450371601577
    '''
    algo = [ Anticor(), BAH(), BCRP(), BestMarkowitz(), BestSoFar(), BNN(), CRP(), CWMR(), DynamicCRP(), 
    EG(), OLMAR(), ONS(),PAMR(),RMR()]
    #noms = ['Anticor', 'BAH', 'BCRP', 'BestMarkowitz', 'BestSoFar', 'BNN', 'CRP', 'CWMR', 'DynamicCRP', 
     #  'EG', 'OLMAR', 'ONS', 'PAMR', 'RMR']
    #d = {noms[i]: algo[i] for i in range(len(algo))}
    df = pd.read_csv('../data/cryptos/année_2022_full_DF.csv', parse_dates = True, index_col = 0)
    #if 'année_2022_full_DF' in path :
    df = df['2022-01-31':'2022-08-07']
    #else : 
    #pass
    #nettoyage
    clo = []
    for e in df.columns :
        if 'Close' in e:
            clo.append(e)
    df = df[clo]
    df = df.dropna(axis = 1)
    result = k.run(df)
    res = result.weights
    # Rendement cumulatif
    df_r = df.copy()[[]]
    for e in df.columns : 
        df_r['rend_cum_' + e[:3]] = df[e]/df.iloc[0][e]
    # Allocation
    allo = df.copy()[[]]
    for e in df :
        allo['allo_' + e[:3]] = df_r['rend_cum_' + e[:3]].values*res[e].values
    pos = df.copy()[[]]
    for e in df.columns : 
        pos['position_' + e[:3]] = allo['allo_' + e[:3]]*m #montant
    pos['Total'] = pos.sum(axis = 1)
    pos['rendement_quotidien'] = pos['Total'].pct_change(1)
    rend_cumulatif = 100*(pos['Total'][-1]/pos['Total'][0] - 1)
    sharp = pos['rendement_quotidien'].mean()/pos['rendement_quotidien'].std()
    sharp_anu = sharp*365**(1/2)
    return sharp_anu, rend_cumulatif

In [78]:
shar = []
metrics = []
algorithme = [ Anticor(), BAH(), BCRP(), BestMarkowitz(), BestSoFar(), BNN(), CRP(), CWMR(), DynamicCRP(), 
    EG(), OLMAR(), ONS(),PAMR(),RMR()]
noms = ['Anticor', 'BAH', 'BCRP', 'BestMarkowitz', 'BestSoFar', 'BNN', 'CRP', 'CWMR', 'DynamicCRP', 
       'EG', 'OLMAR', 'ONS', 'PAMR', 'RMR']
#Scenar_crypto = ['année_2018_DF', 'année_2018_flat_DF','année_2019_flat_DF','année_2021_Nov_DF', 'année_2021_Oct_DF',
#         'covid_DF', 'ukr_war_DF', 'année_2022_full']
#Scenar_crypto_path = ['../data/cryptos/année_2018_DF.csv', '../data/cryptos/année_2018_flat_DF.csv','../data/cryptos/année_2019_flat_DF.csv','../data/cryptos/année_2021_Nov_DF.csv', 
 #                '../data/cryptos/année_2021_Oct_DF.csv', '../data/cryptos/covid_DF.csv', '../data/cryptos/ukr_war_DF.csv','../data/cryptos/année_2022_full_DF.csv'] 
Scenar_crypto_path = ['../data/cryptos/année_2022_full_DF.csv']
Scenar_crypto = ['année_2022_full_DF']
X = pd.DataFrame(index = noms)
for j,i in zip(Scenar_crypto,Scenar_crypto_path) :
    metrics = []
    for algo in algorithme :
        sh = sharp(m = 10000, k = algo)[0]
        metrics.append(sh)
    X[j[:-8]] = metrics
X.loc['PyCrypto'] = sharp_anu
y = X.sort_values(by = 'année_2022', ascending = False)

In [79]:
y

,année_2022
PyCrypto,2.779818
RMR,2.378174
PAMR,2.347056
CWMR,2.343010
OLMAR,2.238710
BNN,1.573680
BestSoFar,0.796291
BCRP,0.795168
BestMarkowitz,0.795167
Anticor,0.670206


In [80]:
shar = []
metrics = []
algorithme = [ Anticor(), BAH(), BCRP(), BestMarkowitz(), BestSoFar(), BNN(), CRP(), CWMR(), DynamicCRP(), 
    EG(), OLMAR(), ONS(),PAMR(),RMR()]
noms = ['Anticor', 'BAH', 'BCRP', 'BestMarkowitz', 'BestSoFar', 'BNN', 'CRP', 'CWMR', 'DynamicCRP', 
       'EG', 'OLMAR', 'ONS', 'PAMR', 'RMR']
#Scenar_crypto = ['année_2018_DF', 'année_2018_flat_DF','année_2019_flat_DF','année_2021_Nov_DF', 'année_2021_Oct_DF',
#         'covid_DF', 'ukr_war_DF', 'année_2022_full']
#Scenar_crypto_path = ['../data/cryptos/année_2018_DF.csv', '../data/cryptos/année_2018_flat_DF.csv','../data/cryptos/année_2019_flat_DF.csv','../data/cryptos/année_2021_Nov_DF.csv', 
 #                '../data/cryptos/année_2021_Oct_DF.csv', '../data/cryptos/covid_DF.csv', '../data/cryptos/ukr_war_DF.csv','../data/cryptos/année_2022_full_DF.csv'] 
Scenar_crypto_path = ['../data/cryptos/année_2022_full_DF.csv']
Scenar_crypto = ['année_2022_full_DF']
X = pd.DataFrame(index = noms)
for j,i in zip(Scenar_crypto,Scenar_crypto_path) :
    metrics = []
    for algo in algorithme :
        sh = sharp(m = 10000, k = algo)[1]
        metrics.append(sh)
    X[j[:-8]] = metrics
X.loc['PyCrypto'] = rend_cumulatif
y = X.sort_values(by = 'année_2022', ascending = False)

In [81]:
y

,année_2022
PyCrypto,17.902754
BestSoFar,17.740655
BNN,17.740655
BCRP,17.740655
BestMarkowitz,17.740644
BAH,-28.973380
EG,-35.716925
CRP,-36.025696
DynamicCRP,-36.025696
OLMAR,-39.835811
